## brightness model

The purpose of this notebook is to model the relationship between features and sky brightness.
Data is based on GaN dataset (which is assumed already written to disk).

See [link](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)


In [19]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

cwd = Path.cwd()
df = pd.read_csv(cwd / "data" / "gan.csv")
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.set_printoptions(sci_mode=False)

In [20]:
features = [
    "Latitude",
    "Longitude",
    "Elevation(m)",
    "CloudCover",
    "UTTimeHour",
    "MoonAlt",
    "MoonAz",
]
feature_tensor = torch.tensor(df[features].values.astype(np.float32))
feature_tensor = torch.nan_to_num(feature_tensor, nan=0.0)

target_tensor = torch.tensor(df["SQMReading"].values.astype(np.float32)).type(
    torch.FloatTensor
)

data_tensor = TensorDataset(feature_tensor, target_tensor)
train_size = int(0.8 * len(data_tensor))
test_size = len(data_tensor) - train_size
train_tensor, test_tensor = random_split(data_tensor, [train_size, test_size])

train_dataloader = DataLoader(dataset=train_tensor, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test_tensor, batch_size=16, shuffle=True)

feature_tensor, target_tensor

(tensor([[    34.2365,   -110.0840,   1964.3800,  ...,      0.5000,
             -59.4212,    301.7643],
         [    33.3369,   -111.4250,    561.7730,  ...,      0.7071,
             -65.7617,    313.6892],
         [    38.8878,   -119.8200,   1466.4800,  ...,      0.9659,
             -63.2626,     47.0922],
         ...,
         [    37.8585,   -122.1440,    345.8890,  ...,      0.9659,
             -46.7966,    283.3446],
         [    47.6102,     20.7281,     91.7003,  ...,     -1.0000,
             -30.8350,    272.9328],
         [    47.6102,     20.7281,     91.7585,  ...,     -0.9659,
             -47.4381,    348.4263]]),
 tensor([17.7800, 20.6700, 21.2400,  ..., 19.5000, 20.6700, 20.9400]))

In [21]:
HIDDEN_SIZE = 64 * 3
OUTPUT_SIZE = 1
FEATURES_SIZE = len(features)


class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(FEATURES_SIZE, HIDDEN_SIZE),
            nn.ReLU(),
            nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE // 2),
            nn.ReLU(),
            nn.Linear(HIDDEN_SIZE // 2, OUTPUT_SIZE),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=192, bias=True)
    (1): ReLU()
    (2): Linear(in_features=192, out_features=96, bias=True)
    (3): ReLU()
    (4): Linear(in_features=96, out_features=1, bias=True)
  )
)

In [22]:
loss_fn = nn.HuberLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
def train_loop(
    data_loader: DataLoader,
    model: NeuralNetwork,
    loss_fn: nn.HuberLoss,
    optimizer: torch.optim.Adam,
):
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(X)
        loss = loss_fn(output.squeeze(), y)
        loss.backward()
        nn.utils.clip_grad.clip_grad_norm(model.parameters(), max_norm=5)
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            size = len(data_loader.dataset)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [24]:
def test_model(data_loader: DataLoader, model: NeuralNetwork, loss_fn: nn.MSELoss):
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for batch, (X, y) in enumerate(data_loader):
            pred = model(X)
            print(f"prediction at {batch} was {pred} for {X} ")
            loss = loss_fn(pred.squeeze(), y)
            test_loss += loss.item() * X.size(0)
        avg_loss = test_loss / len(data_loader.dataset)
        print(f"avg loss in test is {avg_loss}")

In [25]:
epochs = 100
for t in range(epochs):
    print(f"epoch {t+1}")
    train_loop(train_dataloader, model, loss_fn, optimizer)

epoch 1
loss: 29.929743 [   16/11888]


/var/folders/44/3fx8v0096vnd8tkl0kcztg8m0000gn/T/ipykernel_63245/1663130383.py:13: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad.clip_grad_norm(model.parameters(), max_norm=5)


loss: 8.616230 [ 1616/11888]
loss: 8.551602 [ 3216/11888]
loss: 4.605179 [ 4816/11888]
loss: 6.830795 [ 6416/11888]
loss: 5.357975 [ 8016/11888]
loss: 3.091461 [ 9616/11888]
loss: 2.771200 [11216/11888]
epoch 2
loss: 4.570073 [   16/11888]
loss: 2.720142 [ 1616/11888]
loss: 5.229364 [ 3216/11888]
loss: 3.538099 [ 4816/11888]
loss: 3.087979 [ 6416/11888]
loss: 2.748679 [ 8016/11888]
loss: 4.063262 [ 9616/11888]
loss: 2.055276 [11216/11888]
epoch 3
loss: 3.133094 [   16/11888]
loss: 4.150903 [ 1616/11888]
loss: 2.161114 [ 3216/11888]
loss: 3.250015 [ 4816/11888]
loss: 1.172303 [ 6416/11888]
loss: 3.106443 [ 8016/11888]
loss: 2.054630 [ 9616/11888]
loss: 2.812716 [11216/11888]
epoch 4
loss: 2.359510 [   16/11888]
loss: 0.758750 [ 1616/11888]
loss: 3.217537 [ 3216/11888]
loss: 2.844547 [ 4816/11888]
loss: 2.924580 [ 6416/11888]
loss: 3.107777 [ 8016/11888]
loss: 1.615434 [ 9616/11888]
loss: 4.077903 [11216/11888]
epoch 5
loss: 1.629712 [   16/11888]
loss: 2.388338 [ 1616/11888]
loss: 1.829

In [26]:
test_model(test_dataloader, model, loss_fn)

prediction at 0 was tensor([[20.7278],
        [18.0702],
        [22.7958],
        [19.0904],
        [18.9053],
        [20.5069],
        [18.4002],
        [20.5435],
        [18.8893],
        [18.8702],
        [21.1876],
        [21.1418],
        [20.0049],
        [20.5894],
        [19.9335],
        [18.4483]]) for tensor([[    52.9209,     13.5482,     65.7518,      0.0000,     -0.5000,
            -32.1783,    341.0777],
        [   -23.3325,    -69.8395,   1033.0800,      0.0000,      0.5000,
            -46.5877,    139.5169],
        [    48.1739,   -114.3380,   1095.3500,      0.0000,      0.8660,
             29.3133,     92.5079],
        [    32.2085,   -110.7980,    833.8000,      0.0000,      0.7071,
            -58.3614,     82.5184],
        [    54.2825,     10.2441,     38.5618,      6.0000,      0.2588,
            -27.7079,     72.2708],
        [   -30.0161,    -70.6139,    743.9700,      0.0000,      0.5000,
            -37.8022,    179.7877],
        [  

prediction at 20 was tensor([[20.3508],
        [17.9185],
        [20.2045],
        [17.0175],
        [14.6319],
        [21.6474],
        [20.6021],
        [21.6666],
        [18.8849],
        [20.1801],
        [21.0827],
        [20.4613],
        [18.1939],
        [19.0012],
        [17.9052],
        [18.1406]]) for tensor([[    32.1914,   -110.7560,    896.5300,      0.0000,      0.7071,
            -29.1142,    282.9879],
        [    42.3179,    -83.2973,    190.1000,      2.0000,     -0.2588,
            -13.2545,    266.3722],
        [    48.4008,     -2.9073,    172.2000,      0.0000,     -0.8660,
            -32.4489,    294.8745],
        [     4.7405,    -74.0467,   2556.0000,      2.0000,     -0.2588,
            -20.7886,    267.8080],
        [   -33.8812,     18.6792,     70.9200,      0.0000,     -0.7071,
            -46.0986,    164.0579],
        [    36.0534,   -112.0840,   2194.2600,      0.0000,      0.9659,
            -27.2995,    322.9114],
        [ 

In [27]:
torch.save(model.state_dict(), cwd / "model.pth")